In [4]:
import sys  
sys.path.insert(0, '/home/garner1/pipelines/WSI-analysis/SG/pipeline/py')
from graviti import *

from numpy.linalg import norm
import numpy as np
import os
import os.path
from os import path
import sys
import glob
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.use('Agg')
import plotly.graph_objects as go
from plotly.graph_objs import *
import plotly.express as px
import hdbscan
import pandas as pd
import umap
import networkx as nx
from scipy import sparse, linalg
import pickle
from sklearn.preprocessing import normalize, scale

import multiprocessing
from joblib import Parallel, delayed


import warnings
warnings.filterwarnings('ignore')

In [5]:
# Plotly contour visualization
import plotly.graph_objects as go
import pandas as pd
import numpy as np
def plotlyContourPlot(fdf,filename):
    # define the pivot tabel for the contour plot
    table = pd.pivot_table(fdf, 
                           values='field', 
                           index=['x_bin'],
                           columns=['y_bin'],
                           aggfunc=np.sum, # take the mean of the entries in the bin
                           fill_value=None)
    
    fig = go.Figure(data=[go.Surface(z=table.values,
                                     x=table.columns.values, 
                                     y=table.index.values,
                                     colorscale='Jet')])
    fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
    fig.update_layout(title='test', autosize=True,
                      scene_camera_eye=dict(x=1.87, y=0.88, z=-0.64),
                      width=1000, height=1000,
                      margin=dict(l=65, r=50, b=65, t=90)
                    )
    fig.show()
    return

In [6]:
def covd_local(r,A,data,row_idx,col_idx):
    mask = row_idx == r         # find nearest neigthbors
    cluster = np.append(r,col_idx[mask]) # define the local cluster, its size depends on the local connectivity
    C = np.cov(data[cluster,:],rowvar=False)
    iu1 = np.triu_indices(C.shape[1])
    vec = C[iu1]
    return (r,vec)

In [7]:
dirname = '../h5/id_52/' # the path to *features.npz files 
sample = '52' #sys.argv[2]  # the sample id
size = 1000 # number of nuclei, use negative value for full set
nn = 10 # set the number of nearest neighbor in the umap-graph. Will be used in CovD as well

features = ['area',
            'perimeter',
            'solidity',
            'eccentricity',
            'circularity',
            'mean_intensity',
            'cov_intensity']

In [ ]:
counter = 0
for f in glob.glob(dirname+'/*features.npz'): # for every fov
    counter += 1
    if counter == 1:            # set up the data arrays
        data = np.load(f,allow_pickle=True)
        fov = data['fov']
        xy = data['centroids']
        morphology = data['morphology']
    else:                       # update the data arrays
        data = np.load(f,allow_pickle=True)
        fov = np.vstack((fov,data['fov']))
        xy = np.vstack((xy, data['centroids']))
        morphology = np.vstack((morphology, data['morphology']))

# Create dataframes with spatial and morphological measurements
df_fov = pd.DataFrame(data=fov, columns=['fov_row','fov_col']) # field of view dataframe
df_xy = pd.DataFrame(data=xy, columns=['cx','cy'])   # centroid dataframe
df_morphology = pd.DataFrame(data=morphology, columns=['area','perimeter','solidity','eccentricity','circularity','mean_intensity','std_intensity'])

# Concatenate spatial and morphological dataframes
df = pd.concat([df_fov,df_xy, df_morphology],axis=1)

In [ ]:
# filter by percentiles in morphologies (hardcoded in function filtering) and introduce coeff. of var
if size < 0:
    fdf = filtering(df) # filter out extremes in morphology
else:
    fdf = filtering(df).sample(n=size) # filter out morphological outlyers and subsample nuclei

pos = fdf[fdf.columns[2:4]].to_numpy() # Get the positions of centroids 

# Building the UMAP graph
filename = '../py/ID'+str(sample)+'.size'+str(size)+'.nn'+str(nn)+'.graph.npz' # the adj sparse matrix
if path.exists(filename):
    print('The graph already exists')
    A = sparse.load_npz(filename) 
else:
    print('Creating the graph')
    A = space2graph(pos,nn)
    sparse.save_npz(filename, A)
    
filename = '../py/ID'+str(sample)+'.size'+str(size)+'.nn'+str(nn)+'.graph.pickle'    # the networkx obj
if path.exists(filename):    
    print('The network already exists')
    G = nx.read_gpickle(filename)
else:
    print('Creating the network')
    G = nx.from_scipy_sparse_matrix(A, edge_attribute='weight')
    nx.write_gpickle(G, filename)

data = scale(fdf[features].to_numpy(), with_mean=False) #get the morphological data and rescale the data by std 
    
# Parallel generation of the local covd
filename = '../py/ID'+str(sample)+'.size'+str(size)+'.nn'+str(nn)+'.descriptor.pickle'    # the descriptor
if False:#path.exists(filename):    
    print('The descriptor already exists')
    descriptor = pickle.load( open( filename, "rb" ) )
else:
    print('Generating the descriptor')
    num_cores = multiprocessing.cpu_count() # numb of cores
    row_idx, col_idx = A.nonzero() # nonzero entries
    processed_list = Parallel(n_jobs=num_cores)(delayed(covd_local)(r,A,data,row_idx,col_idx) 
                                                            for r in range(A.shape[0])
                                                   )

    # Construct the descriptor array
    descriptor = np.zeros((len(processed_list),processed_list[0][1].shape[0]))
    for r in range(len(processed_list)):
        descriptor[r,:] = processed_list[r][1] # covd descriptors of the connected nodes
    pickle.dump( descriptor, open( filename, "wb" ) )
    
# Construct the local Laplacian
L = nx.laplacian_matrix(G, weight='weight') # get the Laplacian matrix
delta_descriptor = L.dot(descriptor) # get the local differianted descriptor
delta = norm(delta_descriptor,axis=1) # get the norm of the differential field

In [ ]:
N = 10 # number of linear bins for the contour visualization

# Contour visualization
fdf['field'] = delta # define the laplacian field
fdf['x_bin'] = pd.cut(fdf['cx'], N, labels=False) # define the x bin label
fdf['y_bin'] = pd.cut(fdf['cy'], N, labels=False) # define the y bin label

# define the pivot tabel for the contour plot
table = pd.pivot_table(fdf, 
                       values='field', 
                       index=['x_bin'],
                       columns=['y_bin'],
                       aggfunc=np.sum, # take the mean of the entries in the bin
                       fill_value=None)

X=table.columns.values
Y=table.index.values
Z=table.values
Xi,Yi = np.meshgrid(X, Y)

fig, ax = plt.subplots(figsize=(10,10))
cs = ax.contourf(Yi, Xi, Z, 
                 alpha=1.0, 
                 levels=5,
                 cmap=plt.cm.Set2);
cbar = fig.colorbar(cs)
plt.savefig('test_sum.png')

In [ ]:
 plotlyContourPlot(fdf,filename)

In [ ]:
fdf['field'].plot.hist(bins=N)
plt.show()

In [ ]:
count_nuclei = fdf.groupby(['x_bin', 'y_bin']).size().to_numpy()

variability = fdf.groupby(['x_bin', 'y_bin'])['field'].mean().to_numpy()

test_df = pd.DataFrame(list(zip(count_nuclei, variability)),
                  columns=['count','variability'])

In [ ]:
print(test_df.corr(method='pearson'))
test_df.plot.scatter(x='count',y='variability')

In [ ]:
fdf.groupby(['x_bin', 'y_bin']).size().plot.hist(bins=N//10)

In [ ]:
fdf.groupby(['x_bin', 'y_bin'])['field'].mean().plot.hist(bins=N//10)

In [ ]:
import plotly.express as px
fig = px.scatter(fdf, 
                 x="x_bin", 
                 y="y_bin",
                 color="field",
                 size=fdf['field'],
                 opacity=0.5,
                 color_continuous_scale=px.colors.sequential.Viridis)
fig.update_layout(template='simple_white',
                 height=800, width=800)

fig.show()

In [ ]:
# Build figure
fig = go.Figure()

# Add scatter trace with medium sized markers
fig.add_trace(
    go.Scatter(
        mode='markers',
        x=fdf['x_bin'],
        y=fdf['y_bin'],
        marker=dict(
            color=fdf['field'],
            size=fdf['field'],
            opacity=0.5,
        ),
        showlegend=False
    )
)
fig.update_layout(template='simple_white',
                 height=500, width=500)
fig.show()